In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filenameq

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import json
from pathlib import Path
from PIL import Image
import timm
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
from scipy.ndimage import gaussian_filter
import io
import shutil

# ============================================================================
# AUGMENTED DATA GENERATION
# ============================================================================

def generate_augmented_images(real_dir, fake_dir, output_real_dir, output_fake_dir, 
                              num_per_class=250):
    """Generate augmented images and save them"""
    
    print("\n" + "="*60)
    print("GENERATING AUGMENTED IMAGES")
    print("="*60)
    
    # Create output directories
    Path(output_real_dir).mkdir(parents=True, exist_ok=True)
    Path(output_fake_dir).mkdir(parents=True, exist_ok=True)
    
    # Heavy augmentation pipeline
    aug_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(32, scale=(0.7, 1.0)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    ])
    
    def augment_and_save(source_dir, target_dir, num_images, prefix):
        source_images = list(Path(source_dir).glob('*'))
        generated = 0
        
        pbar = tqdm(total=num_images, desc=f"Augmenting {prefix}")
        
        while generated < num_images:
            # Randomly select source image
            img_path = np.random.choice(source_images)
            img = Image.open(img_path).convert('RGB')
            
            # Apply augmentation
            aug_img = aug_transform(img)
            
            # Save with unique name
            save_path = Path(target_dir) / f'aug_{prefix}_{generated:04d}.png'
            aug_img.save(save_path)
            
            generated += 1
            pbar.update(1)
        
        pbar.close()
    
    # Generate augmented real images
    augment_and_save(real_dir, output_real_dir, num_per_class, 'real')
    
    # Generate augmented fake images
    augment_and_save(fake_dir, output_fake_dir, num_per_class, 'fake')
    
    print(f"\n✓ Generated {num_per_class} real and {num_per_class} fake augmented images")


def generate_adversarial_images(model, data_dir_real, data_dir_fake, 
                                output_dir, num_images=250, 
                                epsilon_range=[0.02, 0.03, 0.05, 0.08]):
    """Generate adversarial examples and save them"""
    
    print("\n" + "="*60)
    print("GENERATING ADVERSARIAL EXAMPLES")
    print("="*60)
    
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Load original images
    all_images = list(Path(data_dir_real).glob('*'))[:num_images//2] + \
                 list(Path(data_dir_fake).glob('*'))[:num_images//2]
    
    transform = get_transforms(False)
    device = next(model.parameters()).device
    model.eval()
    
    generated = 0
    
    for img_path in tqdm(all_images, desc="Generating adversarial images"):
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        # Random label (doesn't matter, we just want perturbations)
        label = torch.tensor([0]).to(device)
        
        # Random epsilon
        epsilon = np.random.choice(epsilon_range)
        
        # Generate adversarial example
        adv_img = fgsm_attack(model, img_tensor, label, epsilon)
        
        # Denormalize and save
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(device)
        
        adv_img = (adv_img * std + mean).clamp(0, 1)
        adv_img = (adv_img.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
        
        # Save
        save_path = Path(output_dir) / f'adv_{generated:04d}.png'
        Image.fromarray(adv_img).save(save_path)
        
        generated += 1
    
    print(f"\n✓ Generated {generated} adversarial examples")


# ============================================================================
# DATASET & AUGMENTATION
# ============================================================================

class DeepfakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.images = [(p, 0) for p in Path(real_dir).glob('*')] + \
                      [(p, 1) for p in Path(fake_dir).glob('*')]
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        path, label = self.images[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label


def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(0.2, 0.2, 0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])


# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.3), nn.Linear(256, 128), nn.ReLU(),
            nn.Dropout(0.3), nn.Linear(128, 2)
        )
        
    def forward(self, x):
        x = self.features(x).view(x.size(0), -1)
        return self.classifier(x)


# ============================================================================
# ADVERSARIAL ATTACKS
# ============================================================================

def fgsm_attack(model, images, labels, eps=0.03):
    was_training = model.training
    model.eval()  # Set to eval mode to avoid batch norm issues
    
    images.requires_grad = True
    loss = F.cross_entropy(model(images), labels)
    model.zero_grad()
    loss.backward()
    perturbed = torch.clamp(images + eps * images.grad.sign(), 0, 1).detach()
    
    if was_training:
        model.train()  # Restore training mode
    
    return perturbed


# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================

def train_epoch(model, loader, optimizer, device, use_adv=False, eps_range=[0.02, 0.05]):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        
        # Mix clean and adversarial examples (process entire batch at once)
        if use_adv and np.random.rand() > 0.3:
            eps = np.random.choice(eps_range)
            imgs = fgsm_attack(model, imgs, labels, eps)
        
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, device):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs.to(device))
            probs = F.softmax(outputs, 1)
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds) * 100
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    return acc, f1, auc


# ============================================================================
# ADVERSARIAL EVALUATION & VISUALIZATION
# ============================================================================

def eval_adversarial(model, loader, device, epsilons=[0.0, 0.02, 0.03, 0.05, 0.08]):
    results = {}
    for eps in epsilons:
        all_preds, all_labels = [], []
        for imgs, labels in tqdm(loader, desc=f"ε={eps:.3f}", leave=False):
            imgs = imgs.to(device)
            if eps > 0:
                imgs = fgsm_attack(model, imgs, labels.to(device), eps)
            with torch.no_grad():
                preds = model(imgs).argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
        
        acc = accuracy_score(all_labels, all_preds) * 100
        results[eps] = acc
        print(f"  ε={eps:.3f}: {acc:.2f}%")
    return results


def visualize_adversarial(model, imgs, labels, device, save_path, num_samples=6):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    epsilons = [0.0, 0.02, 0.05, 0.08]
    
    fig, axes = plt.subplots(num_samples, len(epsilons), figsize=(16, 4*num_samples))
    
    for i in range(num_samples):
        img = imgs[i:i+1].to(device)
        label = labels[i].item()
        
        for j, eps in enumerate(epsilons):
            adv_img = fgsm_attack(model, img, labels[i:i+1].to(device), eps) if eps > 0 else img
            
            with torch.no_grad():
                pred = model(adv_img).argmax(1).item()
                prob = F.softmax(model(adv_img), 1)[0].cpu().numpy()
            
            # Denormalize
            vis_img = (adv_img.cpu()[0] * std + mean).clamp(0, 1)
            vis_img = (vis_img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
            
            axes[i, j].imshow(vis_img)
            color = 'green' if pred == label else 'red'
            
            title = f"ε={eps:.2f}\n"
            title += f"{'✓' if pred==label else '✗'} "
            title += f"{'Fake' if pred==1 else 'Real'} ({prob[pred]:.2f})\n"
            title += f"True: {'Fake' if label==1 else 'Real'}"
            
            axes[i, j].set_title(title, color=color, fontweight='bold', fontsize=11)
            axes[i, j].axis('off')
            
            # Add red border for incorrect predictions
            if pred != label:
                for spine in axes[i, j].spines.values():
                    spine.set_edgecolor('red')
                    spine.set_linewidth(4)
                    spine.set_visible(True)
    
    plt.suptitle(f'Adversarial Robustness Visualization', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.close()


def plot_comparison(before, after, save_path):
    eps_list = sorted(before.keys())
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    before_vals = [before[e] for e in eps_list]
    after_vals = [after[e] for e in eps_list]
    
    ax.plot(eps_list, before_vals, 'o-', label='Before Adv. Training', 
            linewidth=3, markersize=10, color='#e74c3c')
    ax.plot(eps_list, after_vals, 's-', label='After Adv. Training', 
            linewidth=3, markersize=10, color='#27ae60')
    
    # Add improvement annotations
    for i, eps in enumerate(eps_list):
        if eps > 0:
            improvement = after_vals[i] - before_vals[i]
            mid_y = (before_vals[i] + after_vals[i]) / 2
            ax.annotate(f'+{improvement:.1f}%', 
                       xy=(eps, mid_y), 
                       fontsize=11, 
                       color='darkgreen', 
                       fontweight='bold',
                       ha='center',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))
    
    ax.set_xlabel('FGSM Epsilon (Perturbation Strength)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
    ax.set_title('Adversarial Robustness: Before vs After Training', 
                fontsize=16, fontweight='bold', pad=20)
    ax.legend(fontsize=13, loc='best', frameon=True, shadow=True)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=1.5)
    ax.set_xticks(eps_list)
    
    # Add horizontal line at 50% (random guess)
    ax.axhline(y=50, color='gray', linestyle=':', linewidth=2, alpha=0.5, label='Random Guess')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.close()


# ============================================================================
# GRAD-CAM
# ============================================================================

class GradCAM:
    def __init__(self, model, layer):
        self.model = model
        self.layer = layer
        self.gradients = None
        self.activations = None
        layer.register_forward_hook(lambda m, i, o: setattr(self, 'activations', o.detach()))
        layer.register_backward_hook(lambda m, gi, go: setattr(self, 'gradients', go[0].detach()))
    
    def generate(self, img, target=None):
        self.model.eval()
        out = self.model(img)
        target = target or out.argmax(1).item()
        self.model.zero_grad()
        out[0, target].backward()
        
        weights = self.gradients[0].mean((1, 2), keepdim=True)
        cam = (weights * self.activations[0]).sum(0)
        cam = F.relu(cam)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam.detach().cpu().numpy()


def visualize_gradcam(model, imgs, labels, device, layer, save_path, num_samples=6):
    gradcam = GradCAM(model, layer)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(14, 4.5*num_samples))
    
    for i in range(num_samples):
        img = imgs[i:i+1].to(device)
        label = labels[i].item()
        
        # Get prediction
        with torch.no_grad():
            pred = model(img).argmax(1).item()
            prob = F.softmax(model(img), 1)[0, pred].item()
        
        # Original
        vis_img = (imgs[i] * std + mean).clamp(0, 1).permute(1, 2, 0).numpy()
        vis_img = (vis_img * 255).astype(np.uint8)
        axes[i, 0].imshow(vis_img)
        axes[i, 0].set_title(f'Original\nTrue: {"Fake" if label==1 else "Real"}', 
                           fontsize=11, fontweight='bold')
        axes[i, 0].axis('off')
        
        # CAM
        cam = gradcam.generate(img)
        cam_resized = cv2.resize(cam, (vis_img.shape[1], vis_img.shape[0]))
        heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        axes[i, 1].imshow(heatmap)
        axes[i, 1].set_title('Grad-CAM Heatmap', fontsize=11, fontweight='bold')
        axes[i, 1].axis('off')
        
        # Overlay
        overlay = (heatmap * 0.5 + vis_img * 0.5).astype(np.uint8)
        axes[i, 2].imshow(overlay)
        color = 'green' if pred == label else 'red'
        axes[i, 2].set_title(f'Overlay\nPred: {"Fake" if pred==1 else "Real"} ({prob:.2f})', 
                           fontsize=11, fontweight='bold', color=color)
        axes[i, 2].axis('off')
    
    plt.suptitle('Grad-CAM Visualization: Model Focus Regions', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.close()


# ============================================================================
# CORRUPTION TESTING
# ============================================================================

def add_corruption(img, corruption_type, severity=3):
    img_np = np.array(img)
    
    if corruption_type == 'gaussian_noise':
        noise = np.random.normal(0, [0.04, 0.06, 0.08][severity-1], img_np.shape)
        img_np = np.clip(img_np + noise * 255, 0, 255)
    elif corruption_type == 'gaussian_blur':
        sigma = [0.6, 0.8, 1.0][severity-1]
        img_np = gaussian_filter(img_np, sigma=sigma)
    elif corruption_type == 'jpeg':
        quality = [65, 50, 40][severity-1]
        buf = io.BytesIO()
        Image.fromarray(img_np.astype(np.uint8)).save(buf, 'JPEG', quality=quality)
        buf.seek(0)
        return Image.open(buf)
    
    return Image.fromarray(img_np.astype(np.uint8))


def test_corruptions(model, loader, device):
    corruptions = ['gaussian_noise', 'gaussian_blur', 'jpeg']
    results = {'clean': validate(model, loader, device)[0]}
    
    for corruption in corruptions:
        all_preds, all_labels = [], []
        for imgs, labels in tqdm(loader, desc=corruption):
            corrupted = []
            for img in imgs:
                vis = (img.permute(1,2,0).numpy() * [0.229,0.224,0.225] + [0.485,0.456,0.406]) * 255
                vis = add_corruption(Image.fromarray(vis.astype(np.uint8)), corruption, 3)
                corrupted.append(get_transforms(False)(vis))
            
            corrupted = torch.stack(corrupted).to(device)
            with torch.no_grad():
                preds = model(corrupted).argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
        
        results[corruption] = accuracy_score(all_labels, all_preds) * 100
        print(f"{corruption}: {results[corruption]:.2f}%")
    
    return results


# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}\n")
    
    # ========================================================================
    # STEP 1: GENERATE AUGMENTED AND ADVERSARIAL DATA
    # ========================================================================
    
    # Generate augmented images (250 real + 250 fake)
    generate_augmented_images(
        real_dir='/kaggle/input/fake-real/data/real_cifake_images',
        fake_dir='/kaggle/input/fake-real/data/fake_cifake_images',
        output_real_dir='data/real_augmented',
        output_fake_dir='data/fake_augmented',
        num_per_class=250
    )
    
    # Merge original and augmented data for training
    print("\nMerging original and augmented data...")
    train_real_dir = Path('data/train_real')
    train_fake_dir = Path('data/train_fake')
    train_real_dir.mkdir(exist_ok=True)
    train_fake_dir.mkdir(exist_ok=True)
    
    # Copy original images
    for img in Path('/kaggle/input/fake-real/data/real_cifake_images').glob('*'):
        shutil.copy(img, train_real_dir / img.name)
    for img in Path('/kaggle/input/fake-real/data/fake_cifake_images').glob('*'):
        shutil.copy(img, train_fake_dir / img.name)
    
    # Copy augmented images
    for img in Path('data/real_augmented').glob('*'):
        shutil.copy(img, train_real_dir / img.name)
    for img in Path('data/fake_augmented').glob('*'):
        shutil.copy(img, train_fake_dir / img.name)
    
    print(f"✓ Training data prepared:")
    print(f"  Real images: {len(list(train_real_dir.glob('*')))}")
    print(f"  Fake images: {len(list(train_fake_dir.glob('*')))}")
    
    # ========================================================================
    # STEP 2: PREPARE DATALOADERS
    # ========================================================================
    
    # Use merged training data
    train_data = DeepfakeDataset(train_real_dir, train_fake_dir, get_transforms(True))
    val_data = DeepfakeDataset('/kaggle/input/fake-real/data/real_cifake_images', '/kaggle/input/fake-real/data/fake_cifake_images', get_transforms(False))
    
    # Larger batch size
    batch_size = 128
    train_loader = DataLoader(train_data, batch_size, shuffle=True, 
                              num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_data, batch_size, num_workers=4, pin_memory=True)
    
    print(f"\nDataset sizes:")
    print(f"  Training: {len(train_data)} images")
    print(f"  Validation: {len(val_data)} images")
    print(f"  Batch size: {batch_size}")
    
    # ========================================================================
    # STEP 3: TRAIN MODELS
    # ========================================================================
    
    # Models including ConvNeXt
    models = {
        'convnext': timm.create_model('convnext_tiny', pretrained=True, num_classes=2),
        'efficientnet': timm.create_model('efficientnet_b0', pretrained=True, num_classes=2),
        'custom_cnn': CustomCNN()
    }
    
    trained_models = []
    num_epochs = 50  # More epochs
    
    for name, model in models.items():
        print(f"\n{'='*70}\nPHASE 1: BASE TRAINING - {name.upper()}\n{'='*70}")
        model = model.to(device)
        
        # Lower learning rate
        optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
        
        best_auc = 0
        patience = 15
        patience_counter = 0
        
        for epoch in range(num_epochs):
            train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
            val_acc, val_f1, val_auc = validate(model, val_loader, device)
            scheduler.step()
            
            print(f"Epoch {epoch+1:2d}/{num_epochs} | "
                  f"Train: {train_acc:.2f}% Loss: {train_loss:.4f} | "
                  f"Val: {val_acc:.2f}% F1: {val_f1:.3f} AUC: {val_auc:.3f}")
            
            if val_auc > best_auc:
                best_auc = val_auc
                patience_counter = 0
                torch.save(model.state_dict(), f'{name}_base.pth')
                print(f"  → New best AUC: {val_auc:.4f} ✓")
            else:
                patience_counter += 1
                
            if patience_counter >= patience:
                print(f"  Early stopping triggered!")
                break
        
        model.load_state_dict(torch.load(f'{name}_base.pth'))
        print(f"\n✓ Base training completed. Best AUC: {best_auc:.4f}")
        
        # ====================================================================
        # GENERATE ADVERSARIAL TRAINING EXAMPLES
        # ====================================================================
        if name == 'efficientnet':  # Use best model to generate adversarial examples
            adv_train_dir = Path('data/adversarial_train')
            generate_adversarial_images(
                model, train_real_dir, train_fake_dir, 
                adv_train_dir, num_images=400,
                epsilon_range=[0.02, 0.03, 0.05, 0.08]
            )
            
            # Add adversarial examples to training data
            print("\nAdding adversarial examples to training data...")
            for img in adv_train_dir.glob('*'):
                # Randomly assign to real or fake folder
                target = train_real_dir if np.random.rand() > 0.5 else train_fake_dir
                shutil.copy(img, target / f'adv_{img.name}')
            
            # Reload training data with adversarial examples
            train_data = DeepfakeDataset(train_real_dir, train_fake_dir, get_transforms(True))
            train_loader = DataLoader(train_data, batch_size, shuffle=True, 
                                     num_workers=4, pin_memory=True)
            print(f"✓ Training data now includes adversarial examples: {len(train_data)} total")
        
        # ====================================================================
        # ADVERSARIAL EVALUATION (BEFORE)
        # ====================================================================
        print(f"\n{'='*70}\nADVERSARIAL ROBUSTNESS TESTING (BEFORE)\n{'='*70}")
        sample_imgs, sample_labels = next(iter(val_loader))
        
        visualize_adversarial(model, sample_imgs[:6], sample_labels[:6], device, 
                            f'{name}_adv_BEFORE.png', num_samples=6)
        before_results = eval_adversarial(model, val_loader, device, 
                                         epsilons=[0.0, 0.02, 0.03, 0.05, 0.08])
        
        # ====================================================================
        # PHASE 2: ADVERSARIAL FINE-TUNING
        # ====================================================================
        print(f"\n{'='*70}\nPHASE 2: ADVERSARIAL FINE-TUNING - {name.upper()}\n{'='*70}")
        
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
        
        for epoch in range(15):  # More fine-tuning epochs
            loss, acc = train_epoch(model, train_loader, optimizer, device, 
                                   use_adv=True, eps_range=[0.02, 0.03, 0.05, 0.08])
            val_acc, val_f1, val_auc = validate(model, val_loader, device)
            print(f"Epoch {epoch+1:2d}/15 | Train: {acc:.2f}% Loss: {loss:.4f} | "
                  f"Val: {val_acc:.2f}% F1: {val_f1:.3f} AUC: {val_auc:.3f}")
        
        torch.save(model.state_dict(), f'{name}_final.pth')
        print(f"\n✓ Adversarial fine-tuning completed")
        
        # ====================================================================
        # ADVERSARIAL EVALUATION (AFTER)
        # ====================================================================
        print(f"\n{'='*70}\nADVERSARIAL ROBUSTNESS TESTING (AFTER)\n{'='*70}")
        
        visualize_adversarial(model, sample_imgs[:6], sample_labels[:6], device, 
                            f'{name}_adv_AFTER.png', num_samples=6)
        after_results = eval_adversarial(model, val_loader, device,
                                        epsilons=[0.0, 0.02, 0.03, 0.05, 0.08])
        
        # Plot comparison
        plot_comparison(before_results, after_results, f'{name}_comparison.png')
        
        # Print improvement summary
        print(f"\n{'='*70}\nIMPROVEMENT SUMMARY - {name.upper()}\n{'='*70}")
        for eps in [0.02, 0.03, 0.05, 0.08]:
            improvement = after_results[eps] - before_results[eps]
            print(f"ε={eps:.3f}: {before_results[eps]:.2f}% → {after_results[eps]:.2f}% "
                  f"({'+'if improvement>0 else ''}{improvement:.2f}%)")
        
        # ====================================================================
        # GRAD-CAM VISUALIZATION
        # ====================================================================
        print(f"\n{'='*70}\nGRAD-CAM VISUALIZATION - {name.upper()}\n{'='*70}")
        
        if name == 'convnext':
            layer = model.stages[-1].blocks[-1]
        elif name == 'efficientnet':
            layer = model.conv_head
        else:
            layer = model.features[-2]
        
        visualize_gradcam(model, sample_imgs[:6], sample_labels[:6], device, layer, 
                         f'{name}_gradcam.png', num_samples=6)
        
        # ====================================================================
        # CORRUPTION TESTING
        # ====================================================================
        print(f"\n{'='*70}\nCORRUPTION ROBUSTNESS TESTING - {name.upper()}\n{'='*70}")
        corruption_results = test_corruptions(model, val_loader, device)
        
        print(f"\nCorruption Results:")
        for corruption, acc in corruption_results.items():
            degradation = corruption_results['clean'] - acc if corruption != 'clean' else 0
            print(f"  {corruption:20s}: {acc:.2f}% (degradation: {degradation:.2f}%)")
        
        trained_models.append(model)
        print(f"\n{'='*70}\n✓ {name.upper()} TRAINING PIPELINE COMPLETED\n{'='*70}")
    
    # ========================================================================
    # ENSEMBLE PREDICTIONS
    # ========================================================================
    print(f"\n{'='*70}\nGENERATING ENSEMBLE PREDICTIONS\n{'='*70}")
    
    test_dir = Path('/kaggle/input/fake-real/data/test')
    predictions = []
    transform = get_transforms(False)
    
    for idx, path in enumerate(tqdm(sorted(test_dir.glob('*')), desc="Predicting")):
        img = transform(Image.open(path).convert('RGB')).unsqueeze(0).to(device)
        
        with torch.no_grad():
            probs = sum(F.softmax(m(img), 1) for m in trained_models) / len(trained_models)
        
        pred = "fake" if probs[0, 1] > 0.5 else "real"
        predictions.append({"index": idx + 1, "prediction": pred})
    
    with open('submission.json', 'w') as f:
        json.dump(predictions, f, indent=4)
    
    print(f"\n{'='*70}")
    print(f"✓ PIPELINE COMPLETED SUCCESSFULLY")
    print(f"{'='*70}")
    print(f"Total predictions: {len(predictions)}")
    print(f"Models trained: {len(trained_models)}")
    print(f"Submission saved: submission.json")
    print(f"{'='*70}\n")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda


GENERATING AUGMENTED IMAGES


Augmenting fake: 100%|██████████| 250/250 [00:02<00:00, 112.70it/s]



✓ Generated 250 real and 250 fake augmented images

Merging original and augmented data...
✓ Training data prepared:
  Real images: 1250
  Fake images: 1250

Dataset sizes:
  Training: 2500 images
  Validation: 2000 images
  Batch size: 128


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

...

gaussian_blur: 63.35%


jpeg: 100%|██████████| 16/16 [00:01<00:00,  8.85it/s]


jpeg: 86.15%

Corruption Results:
  clean               : 90.70% (degradation: 0.00%)
  gaussian_noise      : 84.20% (degradation: 6.50%)
  gaussian_blur       : 63.35% (degradation: 27.35%)
  jpeg                : 86.15% (degradation: 4.55%)

✓ CUSTOM_CNN TRAINING PIPELINE COMPLETED

GENERATING ENSEMBLE PREDICTIONS


Predicting: 100%|██████████| 500/500 [00:10<00:00, 46.38it/s]


✓ PIPELINE COMPLETED SUCCESSFULLY
Total predictions: 500
Models trained: 3
Submission saved: submission.json

